# 07: PyTorch Experiment Tracking

This notebook tries several different models with different types of data augmentation and tracks the results to compare them.

## TODO:
* Setup different experiments
* Track different experiments with SummaryWriter/PyTorch profiler (can view these on Weights & Biases)
    * 1. EffNetB0 + no augmentation
    * 2. EffNetB0 + random augmentation
    * 3. EffNetB4 + no augmentation
    * 4. EffNetB4 + random augmentation
    * Mixed precision?

* See here for more on the profiler: https://pytorch.org/blog/introducing-pytorch-profiler-the-new-and-improved-performance-tool/
* See here on setting up TensorBoard in VSCode: https://code.visualstudio.com/docs/datascience/pytorch-support